<a href="https://colab.research.google.com/github/cristiandsc/taxi-tip-classifier/blob/main/00_nyc_taxi_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificador de Propinas para Viajes en Taxi en NYC (2020)


## Cargando las librerías necesarias

In [2]:
# permitir importar desde src/
import sys, os
sys.path.append(os.path.join(os.getcwd(), "src"))

from data.dataset import load_data
import pandas as pd  # por si quieres hacer df.head(), shape, etc.


## Leemos los datos de enero 2020 (entrenamiento)

In [ ]:
URL_JAN = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2020-01.parquet"
df_jan = load_data(URL_JAN, filetype="parquet")
df_jan.head()
